In [ ]:
import os

from dotenv import load_dotenv

if os.getcwd().endswith("notebooks"):
    os.chdir("./..")
print("Current Directory:", os.getcwd())
load_dotenv()

In [ ]:
import sqlite3
import pandas as pd

conn = sqlite3.connect("./data/transactions.db")
display(pd.read_sql("""SELECT * FROM transactions""", conn))


In [3]:
from bot.pluto import build_pluto_chain
from bot.sqldb import get_db
from bot.utils import get_llm_chat_model
from bot.prompts import (
    DESCRIPTION__SPEND_TRACKER_AGGREGATIONS,
    DESCRIPTION__SPEND_TRACKER_CATEGORIES,
)

llm = get_llm_chat_model(endpoint="openai", model_name="gpt-3.5-turbo", temperature=0)

sql_db = get_db(
    "transactions.db",
    "./data",
    additional_table_info=dict(
        spend_tracker_aggregations=DESCRIPTION__SPEND_TRACKER_AGGREGATIONS,
        spend_tracker_categories=DESCRIPTION__SPEND_TRACKER_CATEGORIES,
    ),
)

DB Dialect: sqlite
DB Usable Table Names: ['spend_tracker_aggregations', 'spend_tracker_categories']


In [43]:
q1 = dict(question="What are my transactions in the 1st quarter?")
result = pluto_chain.invoke(q1)
print("-" * 100)
print("QUESTION:")
print(q1)
print("ANSWER:")
print(result["answer"])
print("QUERY:")
print(result["sql_query"])

----------------------------------------------------------------------------------------------------
QUESTION:
{'question': 'What are my Request Money Payment transactions in the 1st quarter?'}
ANSWER:
Hey there! Your Request Money Payment transactions in the 1st quarter amounted to a total of PHP 2,000,000.13. It looks like you had quite a number of transactions during that period. If you need more details or have any other questions, feel free to ask!
QUERY:
SELECT "transaction_total_amount", "transaction_type", "usecase", "category"
FROM "spend_tracker_aggregations" AS sta
JOIN "spend_tracker_categories" AS stc ON sta."product_code" = stc."product_code" AND sta."event_code" = stc."event_code"
WHERE "transaction_type" = 'DEBIT' 
AND "category" = 'Request Money Payment' 
AND "transaction_month" IN ('01', '02', '03')
AND "transaction_year" = (CASE 
                            WHEN CAST(strftime('%m', date('now')) AS INTEGER) >= 4 THEN strftime('%Y', date('now'))
                       